In [1]:
from tqdm import tqdm
import json
import numpy as np
import copy

import spot_utils
import synthTL

spot_utils.global_lcc = None

In [2]:
translate_dict = synthTL.create_translate_dict('rawcontext_decomposition-worker.xlsx')
cur_nl_spec_fname = "specs/amba_worker.txt"
test_str = open(cur_nl_spec_fname,"r").read()
formula_DUT = translate_dict[test_str]
cur_graph = synthTL.Node(test_str)
synthTL.cur_DUT_variables = spot_utils.get_variables(formula_DUT)

In [3]:
exp_name = "ambaworker_llm-3"

In [4]:
def load_llm_translation(exp_name):
    cur_retranslation_dict = json.load(open(exp_name+"_translations.json"))
    cur_redecomposition_dict = json.load(open(exp_name+"_decompositions.json"))
    all_retranslation_dict = json.load(open(exp_name+"_alltranslations.json"))
    all_redecomposition_dict = json.load(open(exp_name+"_alldecompositions.json"))
    correct_abs_id_list = json.load(open(exp_name+"_correctednodeids.json"))
    return cur_retranslation_dict, cur_redecomposition_dict, all_retranslation_dict, all_redecomposition_dict, correct_abs_id_list

In [5]:
def set_translation_ambiguity(cur_graph,retranslation_dict,redecomposition_dict,all_retranslation_dict,correct_abs_id_list,k_ambiguity = 1,total_ambiguity_threshold = 2**14):
    np.random.seed(0)
    order_idx = np.random.choice(len(correct_abs_id_list),size=len(correct_abs_id_list),replace=False)
    cur_retranslation_dict = copy.deepcopy(retranslation_dict)
    cur_ambiguity = synthTL.count_graphs(cur_graph,cur_retranslation_dict,redecomposition_dict,mode='baseline')
    for next_id_query in [correct_abs_id_list[idx] for idx in order_idx]:
        if len(cur_retranslation_dict[next_id_query]) < k_ambiguity and cur_ambiguity*k_ambiguity < total_ambiguity_threshold:
            cur_translation_list = synthTL.remove_duplicate_translations(all_retranslation_dict[next_id_query])
            extra_select = cur_translation_list[:k_ambiguity-1]
            cur_retranslation_dict[next_id_query] = retranslation_dict[next_id_query] + extra_select
            cur_ambiguity = synthTL.count_graphs(cur_graph,cur_retranslation_dict,redecomposition_dict,mode='baseline')
    print(cur_ambiguity)
    return cur_retranslation_dict
    
cur_retranslation_dict, cur_redecomposition_dict, all_retranslation_dict, all_redecomposition_dict, correct_abs_id_list = load_llm_translation(exp_name)
cur_retranslation_dict = set_translation_ambiguity(
                            cur_graph,
                            cur_retranslation_dict,
                            cur_redecomposition_dict,
                            all_retranslation_dict,
                            correct_abs_id_list,
                            k_ambiguity = 2,
                            total_ambiguity_threshold = 2**13)

4096


In [6]:
all_graph_list = synthTL.get_deconstrain_decomposition_for_node(cur_graph,cur_graph,cur_retranslation_dict,cur_redecomposition_dict,
                                                  cur_cache={},
                                                  skip_set=set(),
                                                  visited_set=set(),
                                                  check_trivial=False,
                                                  filter_res=False,
                                                  mode="all",
                                                   )
len(all_graph_list)

4096

In [7]:
np.random.seed(0)
num_ex = 10
select_idx = np.random.choice(len(all_graph_list),size=np.minimum(num_ex,len(all_graph_list)),replace=False)
cur_graph_list = [all_graph_list[idx] for idx in select_idx]

In [8]:
translation_list = [g.translation for g in cur_graph_list]

run exhaustive model checking

In [9]:
%%time
exh_hold_list = []
for translation in tqdm(translation_list):
    if spot_utils.check_formula_contains_formula(translation,formula_DUT):
        exh_hold_list.append(translation)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:50<00:00, 11.06s/it]

CPU times: user 1min 50s, sys: 1.37 s, total: 1min 51s
Wall time: 1min 50s


run synthTL's batch model checking

In [10]:
%%time
batch_hold_list = synthTL.batch_model_check(translation_list,formula_DUT)

CPU times: user 5.92 s, sys: 92.6 ms, total: 6.01 s
Wall time: 5.94 s


In [11]:
assert set(exh_hold_list) == set(batch_hold_list)